<a href="https://colab.research.google.com/github/ruitenbeek/thesis/blob/main/1step_bertje.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install simpletransformers

     |████████████████████████████████| 225kB 31.1MB/s 
     |████████████████████████████████| 3.3MB 51.6MB/s 
     |████████████████████████████████| 2.3MB 45.4MB/s 
     |████████████████████████████████| 81kB 12.2MB/s 
     |████████████████████████████████| 51kB 8.8MB/s 
     |████████████████████████████████| 1.8MB 35.3MB/s 
     |████████████████████████████████| 225kB 56.1MB/s 
     |████████████████████████████████| 8.2MB 32.8MB/s 
     |████████████████████████████████| 1.2MB 59.6MB/s 
     |████████████████████████████████| 122kB 61.9MB/s 
     |████████████████████████████████| 901kB 50.6MB/s 
     |████████████████████████████████| 102kB 14.0MB/s 
     |████████████████████████████████| 133kB 58.8MB/s 
     |████████████████████████████████| 174kB 61.5MB/s 
     |████████████████████████████████| 245kB 57.1MB/s 
     |████████████████████████████████| 112kB 59.6MB/s 
     |████████████████████████████████| 112kB 60.9MB/s 
     |████████████████████████████████| 81kB 13.1MB

In [ ]:
!nvidia-smi

Tue May 25 20:48:58 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import os
os.chdir('/content/gdrive/My Drive/thesis/code')
!pwd

/content/gdrive/My Drive/thesis/code


#Import

In [ ]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
from csv import DictReader
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report

#Read Files

In [ ]:
def read_file(file):
    data = list()
    abu_count = 0
    off_count = 0
    not_count = 0
    with open(file, 'r') as f:
        reader = DictReader(f, delimiter='\t')
        for row in reader:
            if (row['abusive'] == 'NOT' or row['abusive'] == 'UNKNOWN') and (row['explicitness'] == 'IMPLICIT' or row['explicitness'] == 'EXPLICIT'):
                data.append([row['text'], 'OFF'])
                off_count += 1
            elif row['abusive'] == 'IMPLICIT' or row['abusive'] == 'EXPLICIT':
                data.append([row['text'], 'ABU'])
                abu_count += 1
            elif (row['abusive'] == 'NOT' or row['abusive'] == 'UNKNOWN') and row['explicitness'] == 'NOT':
                data.append([row['text'], 'NOT'])
                not_count += 1
    print(f'ABU: %i\nOFF: %i\nNOT: %i' % (abu_count, off_count, not_count))
    data_df = pd.DataFrame(data)
    data_df.columns = ['text', 'label']
    return data_df


#Model

In [ ]:
def bertje(train_df):
  model_args = ClassificationArgs(num_train_epochs=4, train_batch_size=64, learning_rate=5e-5, manual_seed=0, overwrite_output_dir=True, output_dir='/outputs1')
  model = ClassificationModel('bert', 'GroNLP/bert-base-dutch-cased', 
                              args=model_args, use_cuda=True, num_labels=3)
  model.train_model(train_df)
  return model

#Evaluation

In [ ]:
def evaluation(model, test_df):
    result, model_outputs, wrong_predictions = model.eval_model(test_df, classification_report=classification_report)
    print(result['classification_report'])

# Main

In [ ]:
print('###TRAIN Split###')
train_data = read_file('train_final_pp.csv')
print('\n###DEV Split###')
dev_data = read_file('dev_final_pp.csv')
print('\n###TEST Split###')
test_data = read_file('test_final_pp.csv')
le = LabelEncoder()
train_data['label'] = le.fit_transform(train_data['label'])
dev_data['label'] = le.transform(dev_data['label'])
test_data['label'] = le.transform(test_data['label'])

###TRAIN Split###
ABU: 1143
OFF: 1445
NOT: 5176

###DEV Split###
ABU: 110
OFF: 76
NOT: 361

###TEST Split###
ABU: 637
OFF: 399
NOT: 2072


In [ ]:
for i in range(5):
    print(f'\n###RUN %i###' % i)
    model = bertje(train_data)
    print('\n####DEV RESULTS####')
    evaluation(model, dev_data)
    print('\n####TEST RESULTS####')
    evaluation(model, test_data)


###RUN 0###


Downloading:   0%|          | 0.00/608 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/437M [00:00<?, ?B/s]

Some weights of the model checkpoint at GroNLP/bert-base-dutch-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at GroNLP/bert-base

Downloading:   0%|          | 0.00/242k [00:00<?, ?B/s]

...

Running Evaluation:   0%|          | 0/69 [00:00<?, ?it/s]

              precision    recall  f1-score   support

         0.0       0.66      0.57      0.61       110
         1.0       0.90      0.89      0.90       361
         2.0       0.30      0.38      0.34        76

    accuracy                           0.76       547
   macro avg       0.62      0.61      0.62       547
weighted avg       0.77      0.76      0.76       547


####TEST RESULTS####


  0%|          | 0/3108 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/389 [00:00<?, ?it/s]

              precision    recall  f1-score   support

         0.0       0.72      0.43      0.54       637
         1.0       0.88      0.92      0.90      2072
         2.0       0.35      0.50      0.41       399

    accuracy                           0.77      3108
   macro avg       0.65      0.62      0.62      3108
weighted avg       0.78      0.77      0.76      3108

